<a href="https://colab.research.google.com/github/taavip/automatic-control-of-electrical-equipment-basis-of-price/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests  # Import the requests library

# Query URL
url = ('https://dashboard.elering.ee/api/nps/price?start=2020-05-31T20%3A59%3A59.999Z&end=2020-06-30T20%3A59%3A59.999Z'
       )
# Some api's will have nicer syntax like:
# `&time=2010..2012` or `&na_item=B1GQ,D21`
print(url)

response = requests.get(url)  # Make a GET request to the URL

# Print status code (and associated text)
print(f"Request returned {response.status_code} : '{response.reason}'")

# Print data returned (parsing as JSON)
payload = response.json()  # Parse `response.text` into JSON

import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(payload)
# NOTE: Could use print(response.json()) but this wouldn't be formatted nicely

https://dashboard.elering.ee/api/nps/price?start=2020-05-31T20%3A59%3A59.999Z&end=2020-06-30T20%3A59%3A59.999Z
Request returned 200 : 'OK'
{'data': {'ee': [{'price': 3.88, 'timestamp': 1590958800},
                 {'price': 4.54, 'timestamp': 1590962400},
                 {'price': 4.16, 'timestamp': 1590966000},
                 {'price': 4.54, 'timestamp': 1590969600},
                 {'price': 4.56, 'timestamp': 1590973200},
                 {'price': 3.97, 'timestamp': 1590976800},
                 {'price': 5.63, 'timestamp': 1590980400},
                 {'price': 29.49, 'timestamp': 1590984000},
                 {'price': 40.08, 'timestamp': 1590987600},
                 {'price': 43.75, 'timestamp': 1590991200},
                 {'price': 45.4, 'timestamp': 1590994800},
                 {'price': 44.96, 'timestamp': 1590998400},
                 {'price': 45.08, 'timestamp': 1591002000},
                 {'price': 44.89, 'timestamp': 1591005600},
                 {'price': 44

In [5]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [10]:
payload['data']['ee'] # The values returned aren't very informative!

[{'price': 3.88, 'timestamp': 1590958800},
 {'price': 4.54, 'timestamp': 1590962400},
 {'price': 4.16, 'timestamp': 1590966000},
 {'price': 4.54, 'timestamp': 1590969600},
 {'price': 4.56, 'timestamp': 1590973200},
 {'price': 3.97, 'timestamp': 1590976800},
 {'price': 5.63, 'timestamp': 1590980400},
 {'price': 29.49, 'timestamp': 1590984000},
 {'price': 40.08, 'timestamp': 1590987600},
 {'price': 43.75, 'timestamp': 1590991200},
 {'price': 45.4, 'timestamp': 1590994800},
 {'price': 44.96, 'timestamp': 1590998400},
 {'price': 45.08, 'timestamp': 1591002000},
 {'price': 44.89, 'timestamp': 1591005600},
 {'price': 44.46, 'timestamp': 1591009200},
 {'price': 44.14, 'timestamp': 1591012800},
 {'price': 29.02, 'timestamp': 1591016400},
 {'price': 29.03, 'timestamp': 1591020000},
 {'price': 29.09, 'timestamp': 1591023600},
 {'price': 40.05, 'timestamp': 1591027200},
 {'price': 38.51, 'timestamp': 1591030800},
 {'price': 25.92, 'timestamp': 1591034400},
 {'price': 29.08, 'timestamp': 159103800